In [1]:
import numpy as np
import torch
import pyvista as pv

In [10]:
# Load generated fields
filepath = '/home/dana/Documents/CoNFiLD/CoNFiLD/UnconditionalDiffusionTraining_and_Generation/training/NoisyPipe/gen_fields.npy'
data = np.load(filepath)
N_samples = data.shape[0]//40
N_samples

4

In [3]:
# Load non-normalized coordinates
sim = 0
case = 'NoisyPipe'
coords_o_path = 'data/' + case + '/coords_o.npy'
sdf_o_path = 'data/' + case + '/sdf_o.npy'
sdf_o = np.load(sdf_o_path)
sdf_o = sdf_o[sim,:,:]
coords_o = np.load(coords_o_path)

In [14]:
pl = pv.Plotter(shape=(1,4))
tidx = 20
for i in range(N_samples):
    data_d = data[i*40 + tidx,:,:] # decoded latent

    # Reshape for StructuredGrid
    mesh_d = pv.StructuredGrid()
    mesh_d.points = coords_o
    mesh_d.dimensions = (200, 40, 40)  

    # Add scalar field
    mesh_d['u'] = data_d[:,0]
    mesh_d['v'] = data_d[:,1]
    mesh_d['w'] = data_d[:,2]
    mesh_d['p'] = data_d[:,3]
    for scalar in mesh_d.array_names:
        if scalar != 'g' or scalar !='m':
            mesh_d[scalar] = (mesh_d[scalar]/np.max(np.abs(mesh_d[scalar])))
    mesh_d = mesh_d.clip('y')

    pmin = np.min(mesh_d['p'][mesh_d['p']!=0])
    pmax = np.max(mesh_d['p'][mesh_d['p']!=0])

    # Define scalar and value ranges
    scalar = 'w'
    clim_dict = {
        'u': (-1,1),
        'v': (-1,1),
        'w': (-1,1),
        'p': (pmin,pmax),
        'g': (-1,1),
        'm': (0,1),
    }

    # Visualize
    pl.subplot(0,i)
    pl.add_mesh(mesh_d, scalars=scalar,
                clim=clim_dict[scalar],
                # show_scalar_bar=True
                # show_edges=True
                )
    pl.camera_position='zx'
    pl.show_axes()
    # pl.add_scalar_bar( )
    # pl.add_title('Generated')

pl.link_views()
pl.show()


Widget(value='<iframe src="http://localhost:41619/index.html?ui=P_0x791b31830ec0_9&reconnect=auto" class="pyvi…